In [1]:
from langchain import HuggingFaceHub, LLMChain
from dotenv import load_dotenv
from huggingface_hub import InferenceApi
from langchain import PromptTemplate
import pandas as pd

c:\Users\justa\OneDrive\Desktop\Developer\Thesis_ATM_with_LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [4]:

# Load your CSV data
# Replace with your CSV file path
csv_file_path = 'C:\\Users\\justa\\OneDrive\\Desktop\\Developer\\Thesis_ATM_with_LLM\\data\\uc1\\phraseology_errors.csv'
df = pd.read_csv(csv_file_path)
df

,ATC_Command,Label,Correct_ATC_Command
0,G-CD report heading,Correct,NaN
1,"G-CD, heading 350",Correct,NaN
2,G-CD for identification turn left heading 320 ...,Correct,NaN
3,Squawk 7000,Correct,NaN
4,Borton Traffic G-ABCD taxiing for Runway 09 Bo...,Correct,NaN
5,BIGJET 347 RVR 400 metres,Correct,NaN
6,G-CD radar service terminated due radar failure,Correct,NaN
7,Proceed to stand 26 caution jet blast,Correct,NaN
8,"Works 21 stop immediately, aircraft crossing a...",Correct,NaN
9,Tug 9 Ground tow approved from stand 25 to mai...,Correct,NaN


In [5]:
hf_model_repo = "HuggingFaceH4/zephyr-7b-beta"
hub_llm = HuggingFaceHub(repo_id=hf_model_repo)

c:\Users\justa\OneDrive\Desktop\Developer\Thesis_ATM_with_LLM\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
prompt = PromptTemplate(
    input_variables=["command"],
    template="Determine if the following air traffic controller command is correct in terms of phraseology: '{command}'. Respond with 'Correct' or 'Incorrect' only.\n\nResponse:")

In [7]:
hub_chain = LLMChain(prompt=prompt, llm=hub_llm, verbose=True)


In [8]:
correct_count = 0
for index, row in df.iterrows():
    response = hub_chain.run(row['ATC_Command'])
    print(response)
    first_line = response.split('\n')[0].strip()
    final_llm_answer = first_line.rstrip('.')
    if final_llm_answer == row['Label']:
        correct_count += 1

results_df = pd.DataFrame(
    {'Model': [hf_model_repo], 'Correct Answers': [correct_count]})
# results_file = 'evaluation_results.csv'
# results_df.to_csv(results_file, mode='a', index=False,
#                   header=not os.path.exists(results_file))



> Entering new LLMChain chain...
Prompt after formatting:
Determine if the following air traffic controller command is correct in terms of phraseology: 'G-CD report heading'. Respond with 'Correct' or 'Incorrect' only.

Response:

> Finished chain.
 Incorrect. The correct phraseology is 'G-CD, report your heading please'. The phrase 'report heading' is used to request the current heading of an aircraft. The phrase 'report heading' should be followed by 'please' to indicate politeness and professionalism. Therefore, the air traffic controller's command should be 'G-CD, report your heading please'.


> Entering new LLMChain chain...
Prompt after formatting:
Determine if the following air traffic controller command is correct in terms of phraseology: 'G-CD, heading 350'. Respond with 'Correct' or 'Incorrect' only.

Response:

> Finished chain.
 Incorrect. The correct phraseology should be 'G-CD, turn right heading 350 degrees'.


> Entering new LLMChain chain...
Prompt after formatting:

In [9]:
results_df

,Model,Correct Answers
0,HuggingFaceH4/zephyr-7b-beta,7
